In [492]:
%load_ext blackcellmagic

In [495]:
import geopandas as gpd
import pandas as pd
import os
import re
from shapely.geometry import Point, LineString
from fiona.crs import from_epsg
import logging

# configure logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
handler = logging.FileHandler("error_log.log")
handler.setLevel(logging.ERROR)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
handler.setFormatter(formatter)
logger.addHandler(handler)

path_name = os.getcwd()
folder = "June2019"
rails = ["LIRR", "metro_north", "nyc_subway"]


# these are segments that represent unusual service (rush hour etc;)
# and don't appear on MTA map.
segments_to_remove = [
    "E..N55R",
    "E..S56R",
    "E..S04R",
    "E..N05R",
    "N..N20R",
    "N..S20R",
    "2..N03R",
    "2..S03R",
    "4..S01R",
    "4..S02R",
    "4..S03R",
    "4..S13R",
    "4..N01R",
    "4..N02R",
    "4..N03R",
    "4..N13R",
    "4..S40R",
    "5..S18R",
    "5..N18R",
    "5..N13R",
    "5..N06R",
    "5..N07R",
    "5..N20R",
    "5..N22R",
    "5..S06R",
    "5..S07R",
    "5..S15R",
    "5..S21R",
]

# this is to add a 'group' column to use for MTA's subway map-like coloring of the routes
d = {
    "FS": "S",
    "GS": "S",
    "1": "123",
    "3": "123",
    "2": "123",
    "5": "456",
    "4": "456",
    "7": "7",
    "6": "456",
    "A": "ACE",
    "C": "ACE",
    "E": "ACE",
    "B": "BDFM",
    "D": "BDFM",
    "G": "G",
    "F": "BDFM",
    "H": "S",
    "J": "JZ",
    "M": "BDFM",
    "L": "L",
    "N": "NQR",
    "Q": "NQR",
    "R": "NQR",
    "SI": "SIR",
}

# create a dataframe form group dictionary
route_groups = pd.DataFrame(
    [[key, value] for key, value in d.items()], columns=["route_id", "group"]
)


def create_line_shapes(df, x="lon", y="lat"):
    points = [Point(xy) for xy in zip(df[x], df[y])]
    gdf = gpd.GeoDataFrame(df.copy(), geometry=points)
    line_segments = (
        gdf.groupby(["shape_id"])["geometry"]
        .apply(lambda x: LineString(x.tolist()))
        .reset_index()
    )

    gdf_out = gpd.GeoDataFrame(line_segments, geometry="geometry", crs=from_epsg(4269))
    return gdf_out


def make_rail_lines(path_name, folder, rail):
    try:
        routes = pd.read_csv(
            os.path.join(path_name, folder, f"{rail}", "routes.txt"),
            usecols=["route_id", "route_short_name", "route_long_name", "route_color"],
            dtype={"route_id": str},
        )

        routes.rename(
            columns={
                "route_short_name": "route_short",
                "route_long_name": "route_long",
                "route_color": "color",
            },
            inplace=True,
        )

        shapes = pd.read_csv(
            os.path.join(path_name, folder, f"{rail}", "shapes.txt"),
            usecols=["shape_id", "shape_pt_lat", "shape_pt_lon", "shape_pt_sequence"],
            dtype={"shape_id": str},
        )

        shapes.rename(
            columns={"shape_pt_lat": "lat", "shape_pt_lon": "lon"}, inplace=True
        )

        # create new df that doesn't contain unusual service for MTA (applies to subway only)
        shapes = shapes[~shapes["shape_id"].isin(segments_to_remove)]

        shapes.sort_values(["shape_id", "shape_pt_sequence"], inplace=True)

        trips = pd.read_csv(
            os.path.join(path_name, folder, f"{rail}", "trips.txt"),
            usecols=["route_id", "shape_id"],
            dtype={"shape_id": str, "route_id": str},
        )
        trips = trips.drop_duplicates()

        shapes = shapes.merge(
            trips[["route_id", "shape_id"]], on="shape_id"
        ).drop_duplicates()

        if rail == "metro_north":
            # these shape_ids are from
            shapes = shapes.loc[~shapes["shape_id"].isin(["52", "51"])]

        line_segments = creat_line_shapes(shapes)
        lines = line_segments.merge(trips, on="shape_id").dissolve(
            by="route_id", as_index=False
        )

        rail_lines = lines.merge(routes, on="route_id")

        if rail == "nyc_subway":
            rail_lines = rail_lines.merge(
                route_groups, on="route_id"
            )  # table join for groups (subway only)
            # add missing colors for S and SIR lines of the subway
            rail_lines.loc[rail_lines["route_id"] == "FS", "color"] = "808183"
            rail_lines.loc[rail_lines["route_id"] == "H", "color"] = "808183"
            rail_lines.loc[rail_lines["route_id"] == "SI", "color"] = "053159"
            # and make route_short equal to JZ rather than J
            rail_lines.loc[rail_lines["route_id"] == "J", "route_short"] = "JZ"
            rail_lines = rail_lines.drop("shape_id", 1)
        else:
            rail_lines = rail_lines.drop(["shape_id", "route_short"], 1)
        rail_lines["color"] = "#" + rail_lines["color"]
        rail_lines = rail_lines.to_crs(epsg=2263)  # reproject to State Plane
        rail_lines.to_file(
            os.path.join(
                path_name, f"{folder}", "shapes", f"routes_{rail}_{folder}.shp"
            )
        )
        print (f"Created route shapefiles for {rail}")
    except Exception as e:
        logger.exception("Unexpected exception occurred")
        raise

In [496]:
for rail in rails:
    make_rail_lines(rail=rail, path_name=path_name, folder=folder)

Created route shapefiles for LIRR
Created route shapefiles for metro_north
Created route shapefiles for nyc_subway
